In [2]:
from transformers import pipeline

pipe = pipeline("zero-shot-classification", model="tasksource/deberta-small-long-nli")


Device set to use mps:0


In [8]:
text = "one day I will see the world"
candidate_labels = ['travel', 'cooking', 'dancing']
pipe(text, candidate_labels)

{'sequence': 'one day I will see the world',
 'labels': ['travel', 'dancing', 'cooking'],
 'scores': [0.8819178342819214, 0.09866895526647568, 0.019413234665989876]}

In [41]:
import pandas as pd
from src.config import BaseConfig

df = pd.read_json(BaseConfig().processed_data_dir / "final_data_to_train.json")

df = df.sample(50)
df = df.reset_index(drop=True)

In [42]:
df.head(1).T

,0
assignee_level_order,3
jira_key,PRT-265
weeks_since_member_join,28
time_to_complete_hours,1
task_text,**Summary:** Research and analyze MOX as a new...


In [45]:
question = "Is this task easy or hard? "
question += df.loc[0, "task_text"]

labels = ["easy", "hard"]

result = pipe(question, labels)

In [46]:
results = df.apply(lambda x: pipe(question + x["task_text"], labels), axis=1)

In [47]:
results_predictions = [res["labels"][0] for res in results]
confidences = [res["scores"][0] for res in results]

In [48]:
df["results_predictions"] = [labels.index(res) for res in results_predictions]

In [52]:
df[["time_to_complete_hours", "results_predictions"]].corr()

,time_to_complete_hours,results_predictions
time_to_complete_hours,1.000000,0.023471
results_predictions,0.023471,1.000000


In [59]:
from src.modeling.metrics import weighted_accuracy

weighted_accuracy(df["time_to_complete_hours"], df["results_predictions"].apply(lambda x: {0: 4, 1: 8}.get(x)))

np.float64(-0.2670454545454546)